# Loading datasets and model

In [ ]:
!pip install transformers
!pip install --upgrade accelerate

In [ ]:
# Download the ESM-2 model before running
tokenizer_model = "./model_esm2/esm2_t33_650M_UR50D"
base_model = "./model_esm2/esm2_t33_650M_UR50D"

# higher-level
input_training_data = "./input/organismos/bpertussis/train_pseudomonadota_minus_pertussis.csv"
input_test_data = "./input/organismos/bpertussis/test_pseudomonadota_minus_pertussis.csv"
generated_model = "./model_esm2/bpertussis_pseudomonadota_minus_bpertussis"


# Supporting functions

In [ ]:
import torch

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:

# Passing throuth the label region considering the window size on the whole protein
def segmentBycontext(sequences, start, end, classes,  window):

    center = []
    left = []
    right = []
    labels = []
    count=0;
    window_size = window

    for sequence, start, end, label in zip(sequences, start, end, classes):
        length_sequence = len(sequence)
        position=start-1;

        while position < end:

            center_amino = sequence[position]

            if position > window_size:
                left_aminos = sequence[position - window_size:position]

            else:
                left_aminos = sequence[0:position]

            if window_size < (length_sequence - position) :
                right_aminos = sequence[position + 1:position + 1 + window_size]

            else:
                right_aminos = sequence[position + 1:length_sequence]


            center.append(center_amino)
            left.append(left_aminos)
            right.append(right_aminos)
            labels.append(label)

            position = position + 1;
            count = count + 1;


    return center, left, right, labels

In [ ]:
def process_data_for_bert(data, tokenizer):

    sentence_center = list(data["center"])
    sentence_left = list(data["left"])
    sentence_right = list(data["right"])

    left_center_right = []

    for (left, center, right) in zip(sentence_left, sentence_center, sentence_right):
        left_center_right.append(left + center + right)

    y = list(data["label"])

    X_tokenized = tokenizer(left_center_right, padding=True, truncation=True, max_length=512)
    dataset = Dataset(X_tokenized, y)


    return dataset

In [ ]:
def data_processing_pipeline(data, window_size, tokenizer):

    start = list(data["Info_start_pos"])
    end = list(data["Info_end_pos"])

    sequence = data['Info_sequence'].tolist()
    labels = data['Class'].tolist()

    center, left, right, labels =  segmentBycontext(sequence, start, end, labels, window_size)

    sentences = pd.DataFrame({'center':center, 'left':left, 'right':right, 'label':labels})

    dataset = process_data_for_bert(sentences, tokenizer)


    return dataset

# Fine-tune pretrained model

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import AutoModelForSequenceClassification
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef

batch_size=8

tokenizer = AutoTokenizer.from_pretrained(tokenizer_model, do_lower_case=False)
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=2)

model.cuda()

train_dataset = pd.read_csv(input_training_data)
val_dataset = pd.read_csv(input_test_data)

train_dataset = data_processing_pipeline(train_dataset, 512, tokenizer)
val_dataset = data_processing_pipeline(val_dataset, 512, tokenizer)

# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)
    mcc = matthews_corrcoef(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "mcc": mcc}


args = TrainingArguments(
    generated_model,
    report_to="none",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    optim="adamw_torch"
)

def model_init():
    return model

trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# New code - wrap collator in a dictionary
data_collator = trainer.data_collator
trainer.data_collator = lambda data: dict(data_collator(data))
# End new code



Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at ./model_esm2/esm2_t33_650M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-11-ff7ff8948a73>:58: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

# save the fine-tuned model trained on the higher dataset
trainer.save_model(generated_model)